In [11]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax
import re
import pandas as pd

In [12]:
'''
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = re.sub(r'@\S+', '@user', text)
    return new_text
MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)


def sentiment_score(text):
    

    """ 
    Detects the sentiment of a string of text, using the RoBERTa model.
    :param text: string containing tweet text 
    :returns: sentiment score: int = positive - negative
    """
    score_sum = 'NI'
    try:
        # Sentiment detection
        text_processed = preprocess(str(text))
        encoded_input = tokenizer(text_processed, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        score_sum = (scores[2] - scores[0]) # score = positive - negative
    except Exception as e:
        print(e)
        pass

    return score_sum
'''

'\n# Preprocess text (username and link placeholders)\ndef preprocess(text):\n    new_text = re.sub(r\'@\\S+\', \'@user\', text)\n    return new_text\nMODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"\ntokenizer = AutoTokenizer.from_pretrained(MODEL)\n\nmodel = AutoModelForSequenceClassification.from_pretrained(MODEL)\n#model.save_pretrained(MODEL)\n\n\ndef sentiment_score(text):\n    \n\n    """ \n    Detects the sentiment of a string of text, using the RoBERTa model.\n    :param text: string containing tweet text \n    :returns: sentiment score: int = positive - negative\n    """\n    score_sum = \'NI\'\n    try:\n        # Sentiment detection\n        text_processed = preprocess(str(text))\n        encoded_input = tokenizer(text_processed, return_tensors=\'pt\')\n        output = model(**encoded_input)\n        scores = output[0][0].detach().numpy()\n        scores = softmax(scores)\n        score_sum = (scores[2] - scores[0]) # score = positive - negative\n    except Excep

In [13]:
# data source: https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis

valdata = pd.read_csv(r'more_valdata.csv')
valdata

,airline_sentiment,text,roberta_sentiment
0,neutral,@VirginAmerica What @dhepburn said.,0.030810
1,positive,@VirginAmerica plus you've added commercials t...,-0.880323
2,neutral,@VirginAmerica I didn't today... Must mean I n...,-0.118322
3,negative,@VirginAmerica it's really aggressive to blast...,-0.881011
4,negative,@VirginAmerica and it's a really big bad thing...,-0.929868
...,...,...,...
14635,positive,@AmericanAir thank you we got on a different f...,0.583206
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...,-0.912797
14637,neutral,@AmericanAir Please bring American Airlines to...,0.256273
14638,negative,"@AmericanAir you have my money, you change my ...",-0.887614


In [14]:
correct2 = 0
total = 0
posincorrect = 0
negincorrect = 0
negcount = 0
poscount = 0

In [15]:
for index, row in valdata.iterrows():
    total += 1

    score = row['roberta_sentiment']

    actual = row['airline_sentiment']

    if score < -0.2 and actual == 'negative':
        correct2 += 1
        negincorrect = negincorrect + (-0.2 - score)
        negcount += 1
    elif score > 0.2 and actual == 'positive':
        correct2 += 1
        posincorrect = posincorrect + (score - 0.2)
        poscount += 1
    elif -0.2 < score < 0.2 and actual == 'neutral':
        correct2 += 1

print(f'Correct at 0.2: {correct2}')
print(f'Accuracy with 0.2: {(correct2/total)*100}%\n')

print(f'Total: {total}')
print(f'Error when Negative: {negincorrect/negcount}')
print(f'Error when Positive: {posincorrect/poscount}')


Correct at 0.2: 11378
Accuracy with 0.2: 77.71857923497268%

Total: 14640
Error when Negative: 0.5772277137362244
Error when Positive: 0.6818255048618129
